# Music Charts

In [2]:
from IPython.display import display
import threading, logging
import takco

def thread_function():
    tables = takco.TableSet.extract(source = {'wiki': {'pred': "http://www.wikidata.org/prop/direct/P2291", 'sample':5}})

    for table in tables:
        df = takco.extract.htmltables.tableobj_to_dataframe(table)
        display(df)
    
x = threading.Thread(target=thread_function)
x.start()
x.join()

,Source,Rating
0,Aggregate scores,Aggregate scores
1,Metacritic,75/100 (24 reviews)


,Source,Rating
0,Review scores,Review scores
1,The A.V. Club,B+
2,AllMusic,3.5/5 stars
3,Entertainment Weekly,A−
4,Paste,75/100
5,Pitchfork,7.6/10
6,PopMatters,8/10
7,Rolling Stone,4/5 stars
8,Spin,3/5 stars
9,The Times,3/5 stars


,Publisher,Accolade,Year,Rank
0,ABC News,The 50 Best Albums of 2008,2009,46
1,AllMusic,AllMusic's Favorite Rock Albums of 2008,2008,"Unranked, out of 25"
2,Amazon.com editors' picks,Amazon Music: Best of 2008,2009,62
3,The Buffalo News,Best Albums (2000–2010),2010,Honorable mention
4,Chicago Sun-Times ( Jim DeRogatis ),The Best Albums of 2008,2008,2
5,Chicago Tribune ( Greg Kot ),Top Albums of 2008,2008,10
6,Magnet,Magnet's Top 25 Albums of 2008,2008,25
7,No Ripcord,Top 50 Albums of 2008,2008,38
8,Pitchfork Media,The 50 Best Albums of 2008,2008,41
9,Rolling Stone,Best Albums of 2008,2008,42


,Region,Sales charts (2008),Peak position
0,France,Syndicat National de l'Édition Phonographique,127
1,United Kingdom,UK Albums Chart,153


,Region,Sales charts (2009),Peak position
0,Australia,ARIA Albums Chart,66
1,Belgium,Belgian (Wallonian) Albums Chart,58
2,New Zealand,Official New Zealand Music Chart,31
3,United States,US Billboard 200,174


...

,Chart (2010–2019),Position
0,Australia (ARIA),93
1,US Billboard Hot 100,45


,Chart,Position
0,Belgium (Ultratop Flanders),260
1,Belgium (Ultratop Wallonia),134
2,Switzerland (Schweizer Hitparade),704
3,US Billboard Hot 100 (1958-2018),224
4,US Billboard Pop Songs (1992–2017),19


,Region,Certification,Certified units /sales
0,Australia ( ARIA ),7× Platinum,"490,000"
1,Austria ( IFPI Austria),Gold,"15,000 *"
2,Belgium ( BEA ),Platinum,"30,000 *"
3,Canada ( Music Canada ),5× Platinum,"400,000 *"
4,Denmark ( IFPI Denmark ),Gold,"45,000"
5,France ( SNEP ),Platinum,"150,000 *"
6,Germany ( BVMI ),Platinum,"300,000 ^"
7,Italy ( FIMI ),3× Platinum,"90,000 *"
8,Japan ( RIAJ ),Gold,"100,000 ^"
9,Mexico ( AMPROFON ),3× Platinum+Gold,"210,000 *"


,0,1,2,3
0,Streaming,Denmark ( IFPI Denmark ),2× Platinum,"3,600,000"
1,Streaming,Spain ( PROMUSICAE ),2× Platinum,"8,000,000 ^"


,Country,Date,Format,Label,Ref.
0,United States,"October 1, 2012",Mainstream radio,Atlantic Warner Music Group,
1,Italy,"October 2, 2012",Contemporary hit radio,Atlantic Warner Music Group,
2,United States,"October 25, 2012",Rhythmic contemporary,Atlantic,
3,Germany,"October 3, 2012",Digital download,Atlantic WMG,
4,Germany,"November 2, 2012",CD single,Atlantic WMG,
5,Austria,"November 2, 2012",CD single,Atlantic WMG,
6,Switzerland,"November 2, 2012",CD single,Atlantic WMG,
7,Poland,"November 5, 2012",CD single,Warner Music Poland,
8,Japan,"November 21, 2012",CD single,Atlantic WMG,
9,United Kingdom,"November 26, 2012",Digital download,Unknown,
